In [3]:
# Main Game Loop with Levels
import os
import pygame
import Configure_The_Simulation as config
from Simulated_Functions import (
    draw_layout, draw_player, show_game_over, load_player_image,
    show_mini_game, computer_rect, asset_path, Enemy
)

# Define levels: each level has its computer, enemies, and optional custom walls
LEVELS = [
    {
        "computer_pos": (30, 30),
        "enemies": [
            {"x": 150, "y": 100, "movement": "vertical", "speed": 200},
            {"x": 300, "y": 340, "movement": "horizontal", "speed": 400},
        ],
        "walls": None  # Use default walls
    },
    {
        "computer_pos": (500, 500),
        "enemies": [
            {"x": 100, "y": 400, "movement": "horizontal", "speed": 300},
            {"x": 400, "y": 150, "movement": "vertical", "speed": 250},
        ],
        "walls": None
    },
    # Add more levels here
]

def load_level(screen, level_index):
    """Load a level's computer, enemies, and walls."""
    level_data = LEVELS[level_index]
    
    # Computer rectangle
    comp_rect = pygame.Rect(*level_data["computer_pos"], 40, 40)
    
    # Create enemies
    enemies = []
    for e in level_data["enemies"]:
        enemies.append(Enemy(e["x"], e["y"], movement=e["movement"], speed=e["speed"]))
    
    # Walls
    walls = draw_layout(screen, config.wall_color, config.wall_thickness)
    if level_data.get("walls"):
        walls += level_data["walls"]
    
    return comp_rect, enemies, walls

def main():
    pygame.init()
    screen = pygame.display.set_mode((config.WIDTH, config.HEIGHT))
    pygame.display.set_caption("3x3 Adventure Grid")
    clock = pygame.time.Clock()

    # Load assets
    music_file = asset_path("music", "Music_For_Game.mp3")
    if os.path.exists(music_file):
        pygame.mixer.music.load(music_file)
        pygame.mixer.music.play(-1)
        pygame.mixer.music.set_volume(0.75)

    # Load enemy images once
    Enemy.load_images(config.enemy_image_vertical, config.enemy_image_horizontal, config.enemy_image_size)

    # Load player image
    load_player_image()

    # Player starting position
    start_x = config.WIDTH - config.wall_thickness - config.PLAYER_RADIUS
    start_y = config.HEIGHT - config.wall_thickness - config.PLAYER_RADIUS
    player_x, player_y = start_x, start_y

    # Level system
    current_level = 0
    computer_rect_pos, enemies, walls = load_level(screen, current_level)
    computer_cooldown = False
    running = True

    while running:
        dt = clock.tick(config.FPS) / 1000.0

        # Handle input events
        keys = pygame.key.get_pressed()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if keys[pygame.K_ESCAPE]:
                running = False
                show_game_over(screen)

        # Move player
        dx = dy = 0
        if keys[pygame.K_LEFT] or keys[pygame.K_a]: dx = -1
        if keys[pygame.K_RIGHT] or keys[pygame.K_d]: dx = 1
        if keys[pygame.K_UP] or keys[pygame.K_w]: dy = -1
        if keys[pygame.K_DOWN] or keys[pygame.K_s]: dy = 1
        if dx and dy: dx *= 0.7071; dy *= 0.7071

        new_x = player_x + dx * config.PLAYER_SPEED * dt
        new_y = player_y + dy * config.PLAYER_SPEED * dt
        future_rect = pygame.Rect(new_x - config.PLAYER_RADIUS, new_y - config.PLAYER_RADIUS,
                                  config.PLAYER_RADIUS*2, config.PLAYER_RADIUS*2)

        if not any(future_rect.colliderect(w) for w in walls):
            player_x, player_y = new_x, new_y

        # Update enemies
        for enemy in enemies:
            enemy.update(dt, walls)

        # Draw everything
        screen.fill(config.BG_COLOR)
        draw_layout(screen, config.wall_color, config.wall_thickness)
        for enemy in enemies: enemy.draw(screen)
        player_rect = draw_player(screen, player_x, player_y)

        # Draw computer
        pygame.draw.rect(screen, (194, 194, 194), computer_rect_pos)

        # Collision with enemies
        for enemy in enemies:
            enemy_rect = pygame.Rect(enemy.x, enemy.y, enemy.size[0], enemy.size[1])
            if player_rect.colliderect(enemy_rect):
                player_x, player_y = start_x, start_y
                break

        # Collision with computer
        if player_rect.colliderect(computer_rect_pos):
            if not computer_cooldown:
                computer_cooldown = True
                show_mini_game(screen)  # Play mini-game

                # Advance to next level
                current_level += 1
                if current_level >= len(LEVELS):
                    show_game_over(screen)  # All levels complete
                    running = False
                else:
                    player_x, player_y = start_x, start_y
                    computer_rect_pos, enemies, walls = load_level(screen, current_level)
        else:
            computer_cooldown = False

        pygame.display.flip()

    pygame.quit()

if __name__ == "__main__":
    main()